### Relational databases

## SQLite
### Import sqlite3 and connect to the database

In [ ]:
import sqlite3 as sql

# create a connection to the DB
# if the DB does not exist, it will be created
# verify that after this statement you now have the DB file on your system
conn = sql.connect('sqlite_demo.db')

In [ ]:
# get a cursor to the connection
cur = conn.cursor()

### Create some tables and perform basic operations

In [ ]:
# create two tables
cur.execute('''DROP TABLE IF EXISTS Students;''')
cur.execute('''CREATE TABLE Students(
                id INTEGER PRIMARY KEY,
                first_name TEXT NOT NULL,
                last_name TEXT NOT NULL,
                age INTEGER,
                email TEXT NOT NULL UNIQUE);
            ''')
cur.execute('''DROP TABLE IF EXISTS Student_courses;''')
cur.execute('''CREATE TABLE Student_courses(
                id INTEGER PRIMARY KEY,
                student_id INTEGER NOT NULL,
                course_id INTEGER,
                course_name TEXT NOT NULL,
                FOREIGN KEY(student_id) REFERENCES Student(id));
            ''')

# commit the changes to the DB
conn.commit()

In [ ]:
# insert a row
cur.execute('''INSERT INTO Students
                VALUES (1, 'John', 'Doe', 21, 'john@doe.com');''')

In [ ]:
# insert the SAME row again. It should fail because id is defined as a primary key.
cur.execute('''INSERT INTO Students
                VALUES (1, 'John', 'Doe', 21, 'john@doe.com');''')

In [ ]:
# delete a record
cur.execute('''DELETE FROM Students WHERE id=1;''')

In [ ]:
# create a tuple with the students
the_students = (
    (1, 'John', 'Doe', 21, 'john@doe.com'),
    (2, 'Alice', 'Doe', 22, 'alice@doe.com'),
    (3, 'Rose', 'Short', 21, 'rose@short.com')
)

# and a tuple with the courses they follow
the_courses = (
    (1, 3, 102, 'BDB'),
    (2, 1, 110, 'BDP1'),
    (3, 3, 110, 'BDP1')
)

# now use executemany to insert the values into the tables
cur.executemany('''INSERT INTO Students VALUES(?, ?, ?, ?, ?)''', the_students)
cur.executemany('''INSERT INTO Student_courses VALUES(?, ?, ?, ?)''', the_courses)

### Remember to call commit() after altering the DB
Until you call commit() on a certain connection, the operations you have performed have *not* been written to disk.

In [ ]:
conn.commit()

### Query SQLite

In [ ]:
cur.execute('SELECT * FROM Students')

In [ ]:
print(cur.fetchall())

In [ ]:
# if you call fetchall() again, you will get an empty list
print(cur.fetchall())

In [ ]:
cur.execute('SELECT * FROM Student_courses')

In [ ]:
print(cur.fetchall())

In [ ]:
# select records based on some condition
cur.execute('SELECT * FROM Students WHERE id=3')
print(cur.fetchall())

### Closing the connection
Remember to close a connection when you are done operating with the DB.

In [ ]:
conn.close()

### Catching exceptions

In [ ]:
# a better way to handle sqlite calls in python
try:
    conn = sql.connect('sqlite_demo.db')
    cur = conn.cursor()
    print("Connection successful")
    # put here all your DB processing stuff - do not forget to commit after changes to the db
    cur.execute('SELECT * FROM Students')
    print(cur.fetchall())
    # closing a _cursor_ is a good idea to reclaim memory
    cur.close()
except sql.Error as error:
    print("Error in SQLite:", error)
finally:
    # whatever happens, close the connection
    conn.close()

## Perform data analysis with pandas

First, copy the sample data set from GitHub and to put it in your directory.

### Import the SQLite DB into a pandas dataframe

In [ ]:
import pandas as pd
conn = sql.connect('gubbio_env_2018.sqlite')
df = pd.read_sql_query('SELECT * FROM gubbio', conn)
df.info()

In [ ]:
df.head()

### Using df.loc[]

In [ ]:
# use loc to select data based on the index, like this:
df.loc[1:2]

In [ ]:
# use loc to select data based on some condition:
print(df.loc[df.NO2<0, 'NO2'])

In [ ]:
# if you omit the second argument to loc[], you will get all columns
print(df.loc[df['NO2']<0])

In [ ]:
# use loc[] to set the environmental readings to 0 if they are negative

df.loc[df.NO2<0, 'NO2'] = 0
df.loc[df.O3<0, 'O3'] = 0
df.loc[df.PM10<0, 'PM10'] = 0
df.loc[df.PM25<0, 'PM25'] = 0

In [ ]:
# verify that now there are no NO2 readings < 0
print(df.loc[df.NO2<0, 'NO2'])

### Augmenting and plotting data

In [ ]:
# an example of how the to_datetime function works,
# applied to the year, month, day, and hour columns.
# datetime creates a series of values:
pd.to_datetime(df[['year', 'month', 'day', 'hour']])

In [ ]:
# add a 'timerep' columns of type "datetime"
# combining the year, month, day, hour columns of the dataframe.
# check that it was created:
df['timerep'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
df.head()

In [ ]:
# this is the current data frame:
df.info()

In [ ]:
# set the timerep column to be an index
# look what the dataframe looks like:
df.set_index('timerep', inplace=True)
df.info()

In [ ]:
# now plot data
df.plot(y=['NO2'])
df.plot(y=['O3'])

### Resampling data

In [ ]:
# how resample() works, aggregating by day, then computing the mean of the resampled values for some columns.
df.resample('D').mean()[['PM10','PM25','NO2']]

In [ ]:
# this however is not correct, because the mean takes into account also the 0 values we had introduced.
# therefore create and operate on a new data frame, and use np.nan (from numpy) to replace negative values.

import numpy as np
conn = sql.connect('gubbio_env_2018.sqlite')
df_new = pd.read_sql_query('SELECT * FROM gubbio', conn)
# replace negative values with np.nan
df_new.loc[df_new.NO2<0, 'NO2'] = np.nan
df_new.loc[df_new.O3<0, 'O3'] = np.nan
df_new.loc[df_new.PM10<0, 'PM10'] = np.nan
df_new.loc[df_new.PM25<0, 'PM25'] = np.nan
# add the timerep column
df_new['timerep'] = pd.to_datetime(df_new[['year', 'month', 'day', 'hour']])
# set timerep to be the index
df_new.set_index('timerep', inplace=True)
# now resample data
df_new.resample('D').mean()[['PM10','PM25','NO2']]

In [ ]:
# find all days where PM10>50. The following line does the following:
# 1) resample by day
# 2) take the mean
# 3) perform a query with the condition to get only the rows where the PM10 value is > 50
# 4) show only the PM10 column
df_new.resample('D').mean().query('PM10>50')[['PM10']]

In [ ]:
# find all days where PM25>24
df_new.resample('D').mean().query('PM25>24')[['PM25']]

In [ ]:
# you could also save the days where PM25>24 to a dictionary:
bad_pm25 = df_new.resample('D').mean().query('PM25>24').to_dict()
bad_pm25['PM25']

In [ ]:
# find the yearly PM10 mean: resample by year, using the argument 'A'
df_new.resample('A').mean()[['PM10']]

In [ ]:
# get just the first entry:
df_new.resample('A').mean()[['PM10']].head(1)

In [ ]:
# or, you could use loc to get the same
df_new.resample('A').mean().loc['2018-12-31']

In [ ]:
# and you could get just the PM10 value with
df_new.resample('A').mean().loc['2018-12-31'].at['PM10']

In [ ]:
# find the yearly PM25 mean, resampling by year
df_new.resample('A').mean()[['PM25']]

In [ ]:
# again, let's get just the first entry:
df_new.resample('A').mean()[['PM25']].head(1)

### Saving to SQLite and reading the DB back

In [ ]:
# close the previous connection before moving on.
conn.close()

In [ ]:
# write the data frame to a new SQLite database
conn = sql.connect('gubbio_env_2018_custom.sqlite')
df.to_sql('gubbio', conn, if_exists='replace')
conn.close()

In [ ]:
# read it back
conn = sql.connect('gubbio_env_2018_custom.sqlite')
df2 = pd.read_sql('SELECT * FROM gubbio', conn)
conn.close()

In [ ]:
# check what the data frame looks like
df2.info(memory_usage='deep')

In [ ]:
# the x-axis does not look right...
df2.plot(y=['NO2'])

In [ ]:
# write the data frame to a new SQLite database again,
# but this time tell pandas to write an index to the DB
conn = sql.connect('gubbio_env_2018_custom.sqlite')
df.to_sql('gubbio', conn, if_exists='replace', index=True, index_label='timerep')
conn.close()

In [ ]:
# read it back, but this time tell pandas that there is
# an index in the column 'timerep', of type datetime
conn = sql.connect('gubbio_env_2018_custom.sqlite')
df2 = pd.read_sql('SELECT * FROM gubbio', conn, index_col='timerep', parse_dates=['timerep'])
conn.close()

In [ ]:
# check what the data frame looks like now
df2.info(memory_usage='deep')

In [ ]:
# the x-axis is now correct
df2.plot(y=['NO2'])